In [39]:
!pip install tensorflow==2.17.1 tflearn
!pip install notebook
!pip install nltk
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt_tab')
import time
import difflib
import numpy
import pandas
import tensorflow
!pip show tensorflow
import json
import random
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 933.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.12.0
    Uninstalling tensorflow-2.12.0:
      Successfully uninstalled tensorflow-2.12.0


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [40]:
with open ('intents.json') as file:
  data = json.load(file)
try:
  with open('data.pickle', 'rb') as f:
    words, labels, training, output = pickle.load(f)
except:
  words = []
  labels = []
  docs_patt = []
  docs_tag= []
#TOKENISATION & STEMMING
  for intent in data['intents']:
    for pattern in intent['patterns']:
      wrds = nltk.word_tokenize(pattern)
      for item in wrds:
        words.extend(wrds)
        docs_patt.append(wrds)
        docs_tag.append(intent['tag'])
      if intent['tag'] not in labels:
        labels.append(intent['tag'])
  words = [stemmer.stem(w.lower()) for w in words]
  words = sorted(list(set(words)))
  labels = sorted(labels)
  training = []
  output = []
  out_empty = [0 for _ in range(len(labels))]
#BAG OF WORDS - FEATURE ENGINEERING .
  for x, doc in enumerate(docs_patt):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_tag[x])] = 1
    training.append(bag)
    output.append(output_row)
  training = numpy.array(training)
  output = numpy.array(output)
  with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [41]:
!pip show tflearn | grep Location

Location: /usr/local/lib/python3.11/dist-packages


In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import pickle
import numpy

# Try to load data from the pickle file
try:
    with open('data.pickle', 'rb') as f:
        words, labels, training, output = pickle.load(f)
except FileNotFoundError:
    # If the file is not found, initialize training and output
    # (You'll need to define how to generate or load your data here)

    # Example - Replace this with your data loading/generation logic:
    training = numpy.array([[0, 1, 0], [1, 0, 1]])  # Placeholder data
    output = numpy.array([[1, 0], [0, 1]])        # Placeholder data

    print("data.pickle not found. Please run the data preparation script first.")
    # Optionally, you could exit here to prevent further errors:
    # import sys
    # sys.exit(1)

# Define model
model = Sequential([
    Input(shape=(len(training[0]),)),  # Input layer
    Dense(8, activation='relu'),      # First hidden layer
    Dense(8, activation='relu'),      # Second hidden layer
    Dense(len(output[0]), activation='softmax')  # Output layer
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 768       
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 10)                90        
                                                                 
Total params: 930
Trainable params: 930
Non-trainable params: 0
_________________________________________________________________


In [43]:
try:
  model.load('model.tflearn')
except:
  # Replace 'n_epoch' with 'epochs' and remove 'show_metric'
  model.fit(training, output, epochs=1000, batch_size=8)  # Removed show_metric=True
  model.save('model.tflearn')

Epoch 1/1000
28/28 [==============================] - 1s 4ms/step - loss: 2.2938 - accuracy: 0.0495
Epoch 2/1000
28/28 [==============================] - 0s 4ms/step - loss: 2.2465 - accuracy: 0.2207
Epoch 3/1000
28/28 [==============================] - 0s 4ms/step - loss: 2.1998 - accuracy: 0.3964
Epoch 4/1000
28/28 [==============================] - 0s 3ms/step - loss: 2.1437 - accuracy: 0.4505
Epoch 5/1000
28/28 [==============================] - 0s 2ms/step - loss: 2.0724 - accuracy: 0.4730
Epoch 6/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.9854 - accuracy: 0.5225
Epoch 7/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.8767 - accuracy: 0.5946
Epoch 8/1000
28/28 [==============================] - 0s 2ms/step - loss: 1.7546 - accuracy: 0.6351
Epoch 9/1000
28/28 [==============================] - 0s 4ms/step - loss: 1.6216 - accuracy: 0.6351
Epoch 10/1000
28/28 [==============================] - 0s 6ms/step - loss: 1.4830 - accuracy: 0.6486

In [44]:
def bag_of_words(s, words):
  bag=[0 for _ in range(len(words))]
  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]
  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  return numpy.array(bag)

In [45]:
def words_to_list(s):
  a=[]
  ns=""
  s=s+" "
  for i in range(len(s)):
    if s[i]==" ":
      a.append(ns)
      ns=""
    else:
      ns=ns+s[i]
  a=list(set(a))
  return a

In [46]:
#pass the file in this function to create a dictionary of unique vacabulary
def json_to_dict(data_file='intents.json'): # Added data_file argument
  import json  # Import json module here
  with open(data_file) as file: # Open the JSON file inside the function
    data = json.load(file)  # Load data
  dictionary =[]
  fil_dict=[]
  vocalubary=[]
  for i in data["intents"]:
    for pattern in i["patterns"]:
      vocalubary.append(pattern.lower())
  for i in vocalubary:
    dictionary.append(words_to_list(i))
  for i in range(len(dictionary)):
    # Assuming you want to iterate over words in dictionary[i]
    for word in dictionary[i]:
      if word in dictionary[i]: # 'word' is not defined; Assuming it's meant to iterate over words
        fil_dict.append(word)
  return list(set(fil_dict))

#this function checks the spelling in the sentence
chatbot_vocabulary=json_to_dict() # Call json_to_dict with filename or leave empty to use default

In [54]:
def word_checker(s):
  correct_string = ""
  for word in s.casefold().split():
    if word in chatbot_vocabulary:
      suggestion = difflib.get_close_matches(word, chatbot_vocabulary)
      for x in suggestion:
        pass
        if len(suggestion) == 0:
          pass
        else:
          correct_string = correct_string + " " + str(suggestion[0])
      else:
        correct_string = correct_string + " " + str(word)
    return correct_string

In [55]:
!pip install SpeechRecognition
!pip install pyttsx3

In [56]:
!sudo apt-get update && sudo apt-get install espeak espeak-ng

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [57]:
import speech_recognition as sr
r=sr.Recognizer()
import pyttsx3
engine=pyttsx3.init() #This will initialize the pyttsx3 engine with the espeak driver

In [58]:
def bot_speaking(message):
  engine.say(message)
  engine.runAndWait()
  if engine._inLoop:
    engine.endLoop()

In [59]:
def get_input():
  with sr.Microphone() as source:
    bot_speaking("Hey mate say something")
    audio=r.listen(source,timeout=0)
    bot_speaking("Perfect, Thanks!")
  try:
    msg=r.recognize_google(audio)
    print("TEXT: "+msg);
    bot_speaking("you said "+msg)
    return msg
  except:
    bot_speaking("Sorry mate! It's not working")
    pass;

In [66]:
def chat():
  print("Vajra 🤖 :Hi! I am your personal bot. I am here to answer queries on Social Prachar")
  while True:
    inp=input("You: ")
    if inp.lower() == "quit"or inp==None:
      break
    inp_x = word_checker(inp)
    # Reshape the output of bag_of_words to a 2D array with one row
    results = model.predict(numpy.array([bag_of_words(inp_x, words)]))[0]  # Add extra brackets
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if results[results_index] > 0.7:
      for tg in data["intents"]:
        if tg['tag'] == tag:
          responses = tg['responses']
          ms= random.choice(responses)
          print("Vajra 🤖:"+ms)
          bot_speaking(ms)
    else:
      print("Vajra 🤖: Sorry, I don't know how to answer that")
      bot_speaking("Vajra 🤖:Sorry, I don't know how to answer that")
chat()

Vajra 🤖 :Hi! I am your personal bot. I am here to answer queries on Social Prachar
You: bye
1/1 [==============================] - 0s 33ms/step
Vajra 🤖:goodbye! 👋
You: bye bye
1/1 [==============================] - 0s 28ms/step
Vajra 🤖:Come back soon 🤓
You: quit


In [63]:
from tkinter import *
import random
import numpy
import json
import pickle

# Colors and Fonts
BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"
BOT_NAME = "Lalli"

# Load Model and Data (Ensure you have the proper files: model, words, labels, and intents.json)
model = pickle.load(open("data.pkl", "rb"))
words = pickle.load(open("words.pkl", "rb"))
labels = pickle.load(open("labels.pkl", "rb"))
with open("intents.json", "r") as file:
    data = json.load(file)

# Helper Functions
def bag_of_words(sentence, words):
    sentence_words = sentence.split()
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return numpy.array(bag)

def word_checker(sentence):
    # Example of word normalization (you can enhance it with more processing like stemming)
    return sentence.lower()

def bot_speaking(response):
    # Placeholder for any actions during bot response (e.g., text-to-speech)
    print(f"Bot is speaking: {response}")

def get_response(msg):
    inp_x = word_checker(msg)
    results = model.predict([bag_of_words(inp_x, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if results[results_index] > 0.9:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                return random.choice(responses)
    else:
        return "Sorry, I don't know how to answer that yet."

# Chat Application Class
class ChatApplication:
    def _init_(self):
        self.window = Tk()
        self.setup_main_window()

    def run(self):
        self.window.mainloop()

    def setup_main_window(self):
        self.window.title("Chatbot")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)

        # Head Label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR, text="Welcome to Chatbot", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)

        # Divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)

        # Text Widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)

        # Scrollbar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # Bottom Label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # Message Entry Box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self.on_enter_pressed)

        # Send Button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY, command=lambda: self.on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)

    def on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self.insert_message(msg, "You")

    def insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, END)
        msgl = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msgl)
        self.text_widget.configure(state=DISABLED)

        response = get_response(msg)
        msg2 = f"{BOT_NAME}: {response}\n\n"
        bot_speaking(response)
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        self.text_widget.see(END)

if _name_ == "_main_":
    app = ChatApplication()
    app.run()

FileNotFoundError: [Errno 2] No such file or directory: 'data.pkl'

In [64]:
from tkinter import *
import random
import numpy
import json
import pickle

# Colors and Fonts
BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"
BOT_NAME = "Lalli"

# Load Model and Data (Ensure you have the proper files: model, words, labels, and intents.json)
model = pickle.load(open("chatbot_model.pkl", "rb"))
words = pickle.load(open("words.pkl", "rb"))
labels = pickle.load(open("labels.pkl", "rb"))
with open("intents.json", "r") as file:
    data = json.load(file)

# Helper Functions
def bag_of_words(sentence, words):
    sentence_words = sentence.split()
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return numpy.array(bag)

def word_checker(sentence):
    # Example of word normalization (you can enhance it with more processing like stemming)
    return sentence.lower()

def bot_speaking(response):
    # Placeholder for any actions during bot response (e.g., text-to-speech)
    print(f"Bot is speaking: {response}")

def get_response(msg):
    inp_x = word_checker(msg)
    results = model.predict([bag_of_words(inp_x, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    if results[results_index] > 0.9:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                return random.choice(responses)
    else:
        return "Sorry, I don't know how to answer that yet."

# Chat Application Class
class ChatApplication:
    def __init__(self):
        self.window = Tk()
        self.setup_main_window()

    def run(self):
        self.window.mainloop()

    def setup_main_window(self):
        self.window.title("Chatbot")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)

        # Head Label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR, text="Welcome to Chatbot", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)

        # Divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)

        # Text Widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)

        # Scrollbar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)

        # Bottom Label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)

        # Message Entry Box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self.on_enter_pressed)

        # Send Button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY, command=lambda: self.on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)

    def on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self.insert_message(msg, "You")

    def insert_message(self, msg, sender):
        if not msg:
            return

        self.msg_entry.delete(0, END)
        msgl = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msgl)
        self.text_widget.configure(state=DISABLED)

        response = get_response(msg)
        msg2 = f"{BOT_NAME}: {response}\n\n"
        bot_speaking(response)
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        self.text_widget.see(END)

if __name__ == "__main__":
    app = ChatApplication()
    app.run()


FileNotFoundError: [Errno 2] No such file or directory: 'chatbot_model.pkl'